In [5]:
import requests 
from bs4 import BeautifulSoup 

atm_url = 'https://www.trustpilot.com/categories/atm'

response = requests.get(atm_url)

soup = BeautifulSoup(response.content, "html.parser")

best_atm = soup.find_all('div', attrs = {'class': "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"})

In [6]:
import pandas as pd
name = []
trustscore = []
num_review = []
domain = []

for company in best_atm:

    name.append(company.find('p', {'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text)
    
    try:
        trustscore_text = company.find('span', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ'}).text
        trustscore_value = trustscore_text.split()[1]
        trustscore.append(trustscore_value)
    except AttributeError:
        trustscore.append(None)
    
    try:
        num_review_text = company.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).text
        num_review_value = num_review_text.split('|')[1].split()[0]
        num_review.append(num_review_value)
    except AttributeError:
        num_review.append(None)
    
    domain.append(company.find('div', {'class': 'styles_wrapper___E6__ styles_categoriesLabels__FiWQ4 styles_desktop__U5iWw'}).text)

    
atm_df = pd.DataFrame(list(zip(name, trustscore, num_review, domain)), columns=['companyname', 'trustscore', 'totalreviews', 'domain'])

atm_df.to_csv('atm.csv', index=False)

In [7]:
atm_df.head(10)

,companyname,trustscore,totalreviews,domain
0,Evergreen Credit Union,4.9,219,ATM·Financial Institution·Credit Union
1,Liberty First Credit Union,4.8,198,Car Finance and Loan Company·Credit Union·ATM·...
2,Crypto Dispensers,2.4,118,Cryptocurrency Service·Non-Bank Financial Serv...
3,Swadesh,4.3,9,ATM·Bank·Financial Institution·Money Transfer ...
4,Heritage Valley Federal Credit Union,4.1,11,ATM·Bank·Federal Credit Union·Financial Instit...
5,The Payment HQ,3.8,2,Business to Business Service·Payment Service·F...
6,ASDFTEST,3.7,1,ATM
7,Kryptominerstech,3.7,1,Money Order Service·Business Center·Gold Minin...
8,WPC Merchant Services & Credit Card Processing,3.7,1,ATM·Business to Business Service
9,Meriwest Credit Union,3.5,8,Credit Counseling Service·Car Finance and Loan...


In [12]:
#Establish connection with PostgreSQL using psycopg2

import psycopg2
import numpy as np
import psycopg2.extras as extras

#Function to insert values into existing table
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    col = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, col)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn = psycopg2.connect(
    database="atm_scraping", user='postgres', password='postgres', host='127.0.0.1', port='5432'
)

In [13]:
execute_values(conn, atm_df, 'best_atm')

the dataframe is inserted
